CS5010, Summer 2020 Capstone 1 Web Scraper
Data Understanding </H3>
    
Data Features:

___Initial Data Collection___: For the Data Collection we found two websites to scrape. One website was data regarding humidity levels across all states in The United States of America. The other website was COVID-19 Cases by State which included Total, New Cases, and Deaths. The procedure included webscraping these websites, cleaning the data, and merging them into one csv file. This section helped lay the foundation for the data description, analysis and testing/evaluation aspects of this project. </li>

___Data Description___: The Data Description describes the data acquired highlighting the format and source of data. Additionally, the number of cases/variables, with their descriptions will be explained together with necessary information. This will help evaluate whether the data is suitable and a sufficient case for the analysis of interest. 

___Exploratory Data Analysis (EDA)___: EDA will examine the data more closely. It will highlight the first findings and initial hypothesis of the study of interest. Aggregates, Statistical Analysis, Relationships between variables, and Visualizations (graphs and plots) will be included for further analysis and data exploration. 


___Data Testing___: The Data Testing portion will use the data analysis to come up with queries to reveal intersting and useful information about our data and topic of interest. 

___Data Quality___: The Data Quality will list our conclusions. Here we will summarize our findings, explain how these results could be used by others such as government and health officials. Evaluations and improvements to this study project will be described for future use. 
    


__Initial Data Collection Report__



    The first dataset used was put together by the WorldOMeters information group. The dataset is updated daily to reflect the update count of each of the variables of interest. These include the Total Cases, Deaths, Tests, New Deaths, Cases and Active Deaths of the COVID 19 Pandemic. The data can be found at https://www.worldometers.info/coronavirus/country/us/. 
    
    The second dataset used was put together by the USA World Media Group. The dataset rank locations with Average Humidity Data by State. The dataset show cases the State, its population and Average Humidity level in percentage. The data can be found at www.usa.com/rank/us--average-humidity--state-rank.htm. 
    
    The following Data Collection Report is a simple listing of the data sources acquired along with their locations, the procedures used to procure them, and any difficulties encountered. This section will aid both with future replication of this project and with the execution of similar future projects.

__Data Collection:__

1.	Data Source 1: COVID 19 Cases by State in the United States  
2.	Location: https://www.worldometers.info/coronavirus/country/us/
3.	COVID 19 Cases by State: Total Cases, Deaths, New Cases, Deaths and Active Cases from all States in The USA exctracted for analysis. Dataset updates daily.
4.	Method: This dataset shows the Total COVID 19 Cases by State on a daily basis. New cases are updated daily, together with the total deaths, new deaths, and active cases. It contains data from all 50 States showing 4,638,326 total cases with new cases, total deaths, new deaths and active cases being the nested data. 
a.	Cases are reported to Health Officals in every state and county. Data is collected and available for the public to view. Among these cases, deaths are reported too. No demographic or personal information is included. Each case is represented as is with no other information is provided.
b.	The data are contained in the files covid19_states.csv. More details about the contents and use of the file follows.
5.	Obstacles: This is a dataset that is updated daily. As such, due to the daily change it may not be an appropriate dataset to share as data may have been updated from the time the webpage was scraped. However, from daily monitoring of COVID cases this project data analysis would not be affected.  

1.	Data Source 2: United States Average Humidity State Rank  
2.	Location: www.usa.com/rank/us--average-humidity--state-rank.htm. 
3.	United States, Average Humidity Percentage by State: States and Ranked based on their average humidity percetage level in descending order. 
4.	Method: This dataset shows the average humidity level across all states. Population of each state is included as well.  
a.	The data are contained in the covid19_states_humidity.csv. More details about the contents and use of the file follows.
5.	Obstacles: This is a dataset that appears to be updated once a year gathering humidity levels from states across different time periods within a year and finding the average. As such, gathering average levels of humidity could mean the data is measured over time in the same sample group. Hence, we are unsure if the data is from one county or several. 
    
__covid19 states__and__covid19 states humidity__
csv file containing covid19 cases across states and humidity levels are included in the dataset. Humidity levels are consistent with those from the website. Covid cases were obtained and are consistent with data from first week of July. 

__USA States Humidity Levels and COVID19 Cases Merged Data File Structure (covid19_states_humidityandcases.csv)__
All humidity percentage levels by State, and Covid19 cases, deaths are merged and contained in the file covid19_states_humidityandcases.csv. Each line of this file after the header row represents each state, its humidity level, the total number of covid cases, total deaths, new/active cases and new deaths. The following format was applied: 

USAState, TotalCases, NewCases, TotalDeaths, NewDeaths, ActiveCases, AverageHumidity


In [19]:
# imports
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np
import re
import string
from IPython.core.display import HTML

# settings
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# data viz imports
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# building scrapper: writing elegant parsing/scrapping function to parse data from internet
def get_soup(url):
    """Constructs and returns a soup using the HTML content of `url` passed"""
    # initialize a session
    session = requests.Session()
    # make the request
    html = session.get(url)
    # return the soup
    return bs(html.content, "html.parser")
 
def get_tables(soup):
    """Extracts and returns all tables in a soup object"""
    return soup.find_all("table",{"id":"usa_table_countries_today"})

def get_table_headers(table):
    """Given a table soup, returns all the headers"""
    headers = []
    for th in table.find("tr").find_all("th"):
        headers.append(th.text.strip())
    return headers
 
def get_table_rows(table):
    """Given a table, returns all its rows"""
    rows = []
    for tr in table.find_all("tr")[1:]:
        cells = []
        # grab all td tags in this table row
        tds = tr.find_all("td")
        if len(tds) == 0:
            # if no td tags, search for th tags
 
            ths = tr.find_all("th")
            for th in ths:
                cells.append(th.text.strip())
        else:
            # use regular td tags
            for td in tds:
                cells.append(td.text.strip())
        rows.append(cells)
    return rows

def main(url):
    # get the soup
    soup = get_soup(url)
    # extract all the tables from the web page
    tables = get_tables(soup)
    # iterate over all tables
    for i, table in enumerate(tables, start=1):
        # get the table headers
        headers = get_table_headers(table)
        # get all the rows of the table
        rows = get_table_rows(table)
        # save table as csv file
        table_name = f"table-{i}"
        save_as_csv(table_name, headers, rows)    

def save_as_csv(table_name, headers, rows):
    pd.DataFrame(rows, columns=headers).to_csv("data\covid19_states.csv", encoding='utf-8', index=False)

In [27]:
# get data source: df1_covidcases

# step1. instantiating scrapper function        
main("https://www.worldometers.info/coronavirus/country/us/")
# step2. setting up column names: col_list
col_list = ["USAState","TotalCases","NewCases","TotalDeaths","NewDeaths","ActiveCases"]
# step3. reading data in using read_csv, and casting col_list to headers: df1_covidcases
df1_covidcases = pd.read_csv("data/covid19_states.csv", usecols=col_list)


# get data source: df2_humidity

# step1. assign webpage address of interest that we will scrape
url = 'http://www.usa.com/rank/us--average-humidity--state-rank.htm#:~:text=Rank%20Average%20Humidity%20%E2%96%BC%20State%20%2F%20Population%201.,4.%2080.76%25%20Maine%20%2F%201%2C328%2C535%2047%20more%20rows'
# step2. use pandas read_html() tool to read and parse our site. The [0] indicates that we want to grab the first table on the webpage
df2_humidity = pd.read_html(url)[0]
# step3. set first row to headers
df2_humidity = df2_humidity.rename(columns=df2_humidity.iloc[0])
# step 4. drop duplicated header row inplace
df2_humidity.drop(df2_humidity.index[0], inplace=True)
# step 5. split data by the delimeter "/" into: 'State' and 'Population'
df2_humidity[['State','Population']] = df2_humidity['State / Population'].str.split("/",expand=True)
# step 6. drop 'State / Population' column
df2_humidity.drop(columns=['State / Population'], inplace=True)
# step 7. rename columns using dictionary
df2_humidity.rename(columns={'Average Humidity ▼' : 'Average Humidity', 'State' : 'USAState'},inplace=True)
# step 8. write file to .csv
df2_humidity.to_csv("data\covid19_states_humidity.csv", encoding='utf-8', index=False)

In [16]:
# check data frames from scrapper: df1_covidcases & df2_humidity
print(df1_covidcases.head())
print(df2_humidity.head())

        USAState TotalCases NewCases TotalDeaths  NewDeaths ActiveCases
0      USA Total  3,359,838   +4,192     137,436       33.0   1,731,677
1       New York    426,807      NaN      32,393        NaN     227,391
2     California    319,985      NaN       7,030        4.0     227,398
3          Texas    259,465      NaN       3,228        NaN     128,357
4        Florida    254,511      NaN       4,197        NaN     218,244
5     New Jersey    180,672      NaN      15,603        NaN      88,899
6       Illinois    154,094      NaN       7,369        NaN      31,278
7        Arizona    119,930      NaN       2,151        NaN     103,385
8        Georgia    114,401      NaN       2,996        NaN      93,715
9  Massachusetts    111,398      NaN       8,310        NaN       8,741
   Rank Average Humidity        USAState   Population
1    1.           82.01%           Iowa     3,078,116
2    2.           81.86%  New Hampshire     1,321,069
3    3.           81.46%         Alaska       

In [17]:
# check the info for df1_covidcases
# We see that there are some null values that we will need to work out. Aslo we may need to moralize values within each record for leading and lagging spaces.
print(df1_covidcases.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   USAState     64 non-null     object 
 1   TotalCases   64 non-null     object 
 2   NewCases     15 non-null     object 
 3   TotalDeaths  62 non-null     object 
 4   NewDeaths    9 non-null      float64
 5   ActiveCases  64 non-null     object 
dtypes: float64(1), object(5)
memory usage: 3.1+ KB
None


In [18]:
# check the info for df1_covidcases
# We see that the values listed here are a lot more pure. To merge the dataframes we just need to esure primary and foreign keys are consistent on USAState
print(df2_humidity.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 1 to 51
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Rank              51 non-null     object
 1   Average Humidity  51 non-null     object
 2   USAState          51 non-null     object
 3   Population        51 non-null     object
dtypes: object(4)
memory usage: 2.0+ KB
None


In [22]:
# merge wrangled dataframes
# we see that there is an issue with the primary and foreign keys i.e. lagging space or even spelling
test_merge = pd.merge(df1_covidcases,df2_humidity, how='right', on='USAState')
test_merge.head()

,USAState,TotalCases,NewCases,TotalDeaths,NewDeaths,ActiveCases,Rank,Average Humidity,Population
0,Iowa,NaN,NaN,NaN,NaN,NaN,1.,82.01%,"3,078,116"
1,New Hampshire,NaN,NaN,NaN,NaN,NaN,2.,81.86%,"1,321,069"
2,Alaska,NaN,NaN,NaN,NaN,NaN,3.,81.46%,"728,300"
3,Maine,NaN,NaN,NaN,NaN,NaN,4.,80.76%,"1,328,535"
4,North Dakota,NaN,NaN,NaN,NaN,NaN,5.,80.74%,"704,925"


In [26]:
# here we look at value fields for primary keys to see why they are off. We will need to address this in the data preparation.
frames = [df1_covidcases['USAState'], df2_humidity['USAState']]
result = pd.concat(frames).sort_values()
result

19                         Alabama
36                        Alabama 
49                          Alaska
3                          Alaska 
7                          Arizona
14                        Arizona 
30                        Arkansas
31                       Arkansas 
2                       California
9                      California 
24                        Colorado
10                       Colorado 
21                     Connecticut
16                    Connecticut 
38                        Delaware
50                       Delaware 
62           Diamond Princess Ship
40            District Of Columbia
24           District of Columbia 
58                 Federal Prisons
4                          Florida
28                        Florida 
8                          Georgia
39                        Georgia 
60             Grand Princess Ship
52                            Guam
51                          Hawaii
47                         Hawaii 
41                  

In [28]:
# write data to .csv
# write out covid19_states.csv
df1_covidcases.to_csv("data\covid19_states_cases.csv", encoding='utf-8', index=False)

# write out covid19_states_humidity.csv
df2_humidity.to_csv("data\covid19_states_humidity.csv", encoding='utf-8', index=False)